In [2]:
!pip install -U sagemaker


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


### Datasets

In [3]:
import sys

import boto3
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession

sagemaker_session = sagemaker.session.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()

pipeline_session = PipelineSession()
default_bucket = sagemaker_session.default_bucket()
model_package_group_name = f"AbaloneModelPackageGroupName"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
!mkdir -p data

Now, upload the data into the default bucket. You can select our own data set for the input_data_uri as is appropriate.

In [5]:
local_path = "data/abalone-dataset.csv"

s3 = boto3.resource("s3")
s3.Bucket(f"sagemaker-example-files-prod-{region}").download_file(
    "datasets/tabular/uci_abalone/abalone.csv", local_path
)

base_uri = f"s3://{default_bucket}/abalone"
input_data_uri= sagemaker.s3.S3Uploader.upload(
    local_path=local_path,
    desired_s3_uri=base_uri,
)
print(input_data_uri)

s3://sagemaker-ap-northeast-2-532805286864/abalone/abalone-dataset.csv


Download a second dataset for batch transformation after model creation. You can select our own dataset for the batch_data_uri as is appropriate.

In [6]:
local_path = "data/abalone-dataset-batch"

s3 = boto3.resource("s3")
s3.Bucket(f"sagemaker-servicecatalog-seedcode-{region}").download_file(
    "dataset/abalone-dataset-batch", local_path)

base_uri = f"s3://{default_bucket}/abalone"
batch_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_path,
    desired_s3_uri=base_uri,
)
print(batch_data_uri)

s3://sagemaker-ap-northeast-2-532805286864/abalone/abalone-dataset-batch


### Define Parameters to Parameterize Pipeline Execution

The parameters defined in this workflow include:

- processing_instance_count - The instance count of the processing job.
- instance_type - The ml.* instance type of the training job.
- model_approval_status - The approval status to register with the trained model for CI/CD purposes ("PendingManualApproval" is the default).
- input_data - The S3 bucket URI location of the input data.
- batch_data - The S3 bucket URI location of the batch data.
- mse_threshold - The Mean Squared Error (MSE) threshold used to verify the accuracy of a model

In [7]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)

processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.m5.xlarge")
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)
input_data = ParameterString(
    name="inputData",
    default_value=input_data_uri,
)
batch_data = ParameterString(
    name="BatchData",
    default_value=batch_data_uri,
)
mse_threshold = ParameterFloat(name="MseThreshold", default_value=6.0)

### Define a Processing Step for Feature Engineering

In [8]:
!mkdir -p code

In [9]:
%%writefile code/preprocessing.py
import argparse
import os
import requests
import tempfile

import numpy as np
import numpy as pd

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Since we get a headerless CSV file, we specify the column names here.
feature_columns_names = [
    "sex",
    "length",
    "diameter",
    "height",
    "whole_weight",
    "shucked_weight",
    "viscera_weight",
    "shell_weight",
]
label_column = "rings"

feature_columns_dtype = {
    "sex": str,
    "length": np.float64,
    "diameter": np.float64,
    "height": np.float64,
    "whole_weight": np.float64,
    "shucked_weight": np.float64,
    "viscera_weight": np.float64,
    "shell_weight": np.float64,
}
label_column_dtype = {"rings": np.float64}

def merge_two_dicts(x, y):
    z = x.copy()
    z.update(y)
    return z

if __name__ == "__main__":
    base_dir = "/opt/ml/processing"
    
    # 지금은 data가 s3에 있고, processor.run()의 ProcessingInput으로 base_dir로 download
    df = pd.read_csv(
        f"{base_dir}/input/abalone-dataset.csv", 
        header=None,
        names=feature_columns_names + [label_column],
        dtype=merge_two_dicts(feature_columns_dtype, label_column_dtype),
    )
    numeric_features = list(feature_columns_names)
    numeric_features.remove("sex")
    numeric_transformer = Pipeline(
        steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
    )
    
    categorical_features = ["sex"]
    categorical_transformer = Pipeline(
        steps=[
            ("imputer": SimpleImputer(strategy="constant", fill_value="missing")),
            ("onehot": OneHotEncoder(handle_unknown="ignore")),
        ]
    )
    
    preprocess = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features),
        ]
    )
    
    y = df.pop("rings")
    X_pre = preprocess.fit_transform(df)
    y_pre = y.to_numpy().reshape(len(y), 1)
    
    X = np.concatenate((y_pre, X_pre), axis=1)
    
    np.random.shuffle(X)
    train, validation, test = np.split(X, [int(0.7*len(X)), int(0.85*len(X))])
    
    pd.DataFrame(train).to_csv(f"{base_dir}/train/train.csv", header=False, index=False)
    pd.DataFrame(validation).to_csv(
        f"{base_dir}/validation/validation.csv", header=False, index=False
    )
    pd.DataFame(test).to_csv(f"{base_dir}/test/test.csv", header=False, index=False)
    

Overwriting code/preprocessing.py


Next, create an instance of a SKLearnProcessor processor and use that in our ProcessingStep.
You also specify the framework_version to use throughout this notebook.

In [11]:
from sagemaker.sklearn.processing import SKLearnProcessor

framework_version = "1.2-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type="ml.m5.xlarge",
    instance_count=processing_instance_count,
    base_job_name="sklearn-abalone-process",
    role=role,
    sagemaker_session=sagemaker_session,
)

Finally, we take the output of the processor's run method and pass that as arguments to the ProcessingStep.
By passing the pipeline_session to the sagemaker_session, calling .run() does not launch the processing job, it returns the arguments needed to run the job as a step in the pipeline.

Note the "train_data" and "test_data" named channels specified in the output configuration for the processing job. 
Step Properties can be used in subsequent steps and resolve to their runtime values at execution. 
Specifically, this usage is called out when you define the training step.

In [12]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

#  output의 source는 preprocess.py에 명시된 것과 동일해야 함.
# processor_args = sklearn_processor.run(
sklearn_processor.run(
    inputs=[
        ProcessingInput(source=input_data, destination="/opt/ml/processing/input"),
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
    ],
    code="code/preprocessing.py",
)

# step_process = ProcessingStep(name="AbaloneProcess", step_args=processor_args)

INFO:sagemaker:Creating processing-job with name sklearn-abalone-process-2024-04-05-08-57-34-286


TypeError: Object of type ParameterInteger is not JSON serializable